## Install necessary files

In [1]:
!pip install ffprobe-python
!pip install ffmpeg-python

You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.


## import all necessary libraries and get ffmpeg library files

In [2]:
from ffprobe import FFProbe
from os import listdir
from os.path import isfile, join
import ffmpeg
import pathlib

# current folder path
curr_path = pathlib.Path().resolve()

Format to adhere to:
- Video format (container): mp4
- Video codec: h.264
- Audio codec: aac
- Frame rate: 25 FPS
- Aspect ratio: 16:9 
- Resolution: 640 x 360
- Video bit rate: 2 – 5 Mb/s
- Audio bit rate: up to 256 kb/s
- Audio channels: stereo

### Functions to be used

In [3]:
def find_problematic_fields(video_stream, audio_stream):
    '''
    Takes in the ffmpeg stream as input parameter which will be used to filter the video input.
    Only filter for video framerate and resolution will be performed, other field settings will be done when
    the ffmpeg output video file is made.
    
    Returns problematic_fields (String), video stream and audio stream
    '''
    problematic_fields = ""
    # file format setting will be set along with output file
    if file_format != "mp4":
        problematic_fields += "file_format "

    # video codec setting will be set along with output file
    if v_codec_name != "h264":
        problematic_fields += "video_codec "

    # audio codec setting will be set along with output file
    if a_codec_name != "aac":
        problematic_fields += "audio_codec "

    if v_frame_rate != "25":
        problematic_fields += "video_frame_rate "
        # section 11.90 fps of the link provided
        video_stream = ffmpeg.filter(video_stream, 'fps', fps=25, round='near')

    # aspect ratio setting will be set along with output file
    if v_aspect_ratio != "16:9":
        problematic_fields += "aspect_ratio "

    if v_resolution != "640 x 360":
        problematic_fields += "resolution "
        video_stream = ffmpeg.filter(video_stream, 'scale', w='640', h='360')

    # video bitrate setting will be set along with output file
    if v_bitrate < 2 or v_bitrate > 5:
        problematic_fields += "video_bitrate "

    # audio bitrate setting will be set along with output file
    if a_bitrate < 0 and a_bitrate > 256:
        problematic_fields += "audio_bitrate "

    # channels will be set to stereo by default as later on, we will be using the video and audio stream 
    # to generate the output file
    if a_channel_layout != "stereo" or a_channels != 2:
        problematic_fields += "channels_layout "
        
    return problematic_fields, video_stream, audio_stream

### Retrieving files from folder

In [4]:
# get all files that are present in the Exercise3_Films folder in current path directory
files = [f for f in listdir(f"{curr_path}/Exercise3_Films") if isfile(join(f"{curr_path}/Exercise3_Films", f))]

# open the txt file to write the results of the analysis
result_file = open('file_analysis_report.txt', 'w')

for file in files:
    print(f"\nfile: {file}")
    try:
        metadata=FFProbe(f"{curr_path}/Exercise3_Films/{file}")

        # metadata field names can be found in the following link:
        # https://trac.ffmpeg.org/wiki/FFprobeTips
        # print(metadata)

        # retrieve stream information
        video_stream = metadata.streams[0]
        audio_stream = metadata.streams[1]

        # assign stream fields to variables
        file_format = file.split('.')[1]
        v_codec_name = video_stream.codec_name
        a_codec_name = audio_stream.codec_name
        v_frame_rate = float(video_stream.nb_frames) / float(video_stream.duration)
        v_aspect_ratio = video_stream.display_aspect_ratio
        v_resolution = f"{video_stream.width} x {video_stream.height}"
        v_bitrate = int(video_stream.bit_rate) / 1000000
        a_bitrate = int(audio_stream.bit_rate) / 1000
        a_channel_layout = audio_stream.channel_layout
        a_channels = audio_stream.channels

        print(f"Video format (container): {file_format}")
        print(f"Video codec: {v_codec_name}")
        print(f"Audio codec: {a_codec_name}")
        print(f"Frame rate: {format(v_frame_rate, '.2f')} FPS")
        print(f"Aspect ratio: {v_aspect_ratio}")
        print(f"Resolution: {v_resolution}")
        print(f"v_bitrate: {format(v_bitrate, '.2f')}Mb/s")
        print(f"a_bitrate: {format(a_bitrate, '.2f')}kb/s")
        print(f"channel layout: {a_channel_layout}")
        print(f"channels: {a_channels}")
        
        
        # all field filters and conversions are available at: https://ffmpeg.org/ffmpeg-filters.html
        # input current file to ffmpeg input stream
        stream = ffmpeg.input(f"{curr_path}/Exercise3_Films/{file}")
        video_stream = stream.video
        audio_stream = stream.audio
        
        # check if variable fields have the correct format
        # if incorrect, convert them to the correct format
        problematic_fields, video_stream, audio_stream = find_problematic_fields(video_stream, audio_stream)

        # write lines to file_analysis_report.txt based on findings
        # if no problematic fields no files need to be generated
        if problematic_fields == "":
            result_file.write(f"{file} - no issues found\n")
        else:
            result_file.write(f"{file} - {problematic_fields}\n")
            output_filename = f"{curr_path}/OutputFiles/{file.split('.')[0]}_formatOK.mp4"
            stream = ffmpeg.output(video_stream, audio_stream, output_filename, format='mp4', vcodec='h264', acodec='aac', video_bitrate='2.5M', audio_bitrate='256k', aspect='16:9')        
            stream = ffmpeg.run(stream, capture_stdout=True, capture_stderr=True)        

    except ffmpeg.Error as e:
        print('stdout:', e.stdout.decode('utf8'))
        print('stderr:', e.stderr.decode('utf8'))
        raise e

# close file after analysis
result_file.close()


file: Cosmos_War_of_the_Planets.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 29.97 FPS
Aspect ratio: 314:177
Resolution: 628 x 354
v_bitrate: 2.99Mb/s
a_bitrate: 317.10kb/s
channel layout: stereo
channels: 2

file: Last_man_on_earth_1964.mov
Video format (container): mov
Video codec: prores
Audio codec: pcm_s16le
Frame rate: 23.98 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 9.29Mb/s
a_bitrate: 1536.00kb/s
channel layout: stereo
channels: 2

file: The_Hill_Gang_Rides_Again.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 25.00 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 7.54Mb/s
a_bitrate: 253.27kb/s
channel layout: stereo
channels: 2

file: Voyage_to_the_Planet_of_Prehistoric_Women.mp4
Video format (container): mp4
Video codec: hevc
Audio codec: mp3
Frame rate: 29.97 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 8.04Mb/s
a_bitrate: 320.00kb/s
channel layout: stereo
channels: 2

file: 

## Check format of converted files

In [5]:
files = [f for f in listdir(f"{curr_path}/OutputFiles/") if isfile(join(f"{curr_path}/OutputFiles/", f))]

for file in files:
    print(f"file: {file}")
    metadata=FFProbe(f"{curr_path}/OutputFiles/{file}")
    # retrieve stream information
    video_stream = metadata.streams[0]
    audio_stream = metadata.streams[1]

    # assign stream fields to variables
    file_format = file.split('.')[1]
    v_codec_name = video_stream.codec_name
    a_codec_name = audio_stream.codec_name
    v_frame_rate = float(video_stream.nb_frames) / float(video_stream.duration)
    v_aspect_ratio = video_stream.display_aspect_ratio
    v_resolution = f"{video_stream.width} x {video_stream.height}"
    v_bitrate = int(video_stream.bit_rate) / 1000000
    a_bitrate = int(audio_stream.bit_rate) / 1000
    a_channel_layout = audio_stream.channel_layout
    a_channels = audio_stream.channels
    print(f"Video format (container): {file_format}")
    print(f"Video codec: {v_codec_name}")
    print(f"Audio codec: {a_codec_name}")
    print(f"Frame rate: {format(v_frame_rate, '.2f')} FPS")
    print(f"Aspect ratio: {v_aspect_ratio}")
    print(f"Resolution: {v_resolution}")
    print(f"v_bitrate: {format(v_bitrate, '.2f')}Mb/s")
    print(f"a_bitrate: {format(a_bitrate, '.2f')}kb/s")
    print(f"channel layout: {a_channel_layout}")
    print(f"channels: {a_channels}\n")

file: Cosmos_War_of_the_Planets_formatOK.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 25.00 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 2.47Mb/s
a_bitrate: 245.59kb/s
channel layout: stereo
channels: 2

file: Last_man_on_earth_1964_formatOK.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 25.00 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 2.57Mb/s
a_bitrate: 240.95kb/s
channel layout: stereo
channels: 2

file: The_Gun_and_the_Pulpit_formatOK.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 25.00 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 2.41Mb/s
a_bitrate: 251.32kb/s
channel layout: stereo
channels: 2

file: Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
Video format (container): mp4
Video codec: h264
Audio codec: aac
Frame rate: 25.00 FPS
Aspect ratio: 16:9
Resolution: 640 x 360
v_bitrate: 2.38Mb/s
a_bitrate: 246.17kb/s
channel layout: stereo